In [61]:
from pathlib import Path

parent_path = str(Path(__name__).resolve().parents[2]) + '/data/processed/'

model_parent_path = str(Path(__name__).resolve().parents[2]) + '/data/'
test_data_directory = parent_path + 'test'
test_data_name = "test_dataset.csv"
pipe_directory = parent_path + 'pipe'
pipe_name = "pipe.joblib"
model_directory = model_parent_path + 'model'
onnx_model_name = "machine_rf_0.onnx"
sk_model_name = "machine_rf_0.joblib"

In [62]:
test_data_path = os.path.join(test_data_directory, test_data_name)
test_data = pd.read_csv(test_data_path)

In [89]:
X_test = test_data.drop(labels=["machine_failure"], axis=1)
y_test = test_data["machine_failure"]

In [90]:
model_path = os.path.join(model_directory, onnx_model_name)

In [91]:
pipe_path = os.path.join(pipe_directory, pipe_name)

In [92]:
pipe = joblib.load(pipe_path)

In [93]:
import onnxruntime as rt

sess = rt.InferenceSession(model_path, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
input_name, label_name

('float_input', 'output_label')

In [94]:
inputs = pipe.transform(X_test).to_numpy().astype(np.float32)

UDI 는 포함되지 않은 인덱스입니다.


In [165]:
pred_onx = sess.run([label_name], {input_name: inputs})[0]

In [166]:
pred_onx

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [162]:
sess.run([label_name], {input_name: inputs})[1]

IndexError: list index out of range

In [ ]:
import time
import pandas as pd
import numpy as np


sess = rt.InferenceSession(model_path, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
label = {
    0: "normal",
    1: "failure"
}

labels = []
predicted = []
trans_durations = []
infer_durations = []
for i, row in X_test.iterrows():
    x = (pd.DataFrame([row]))
    trans_start = time.time()
    x_trans = pipe.transform(x).to_numpy().astype(np.float32)
    trans_end = time.time()
    pred_onx = sess.run([label_name], {input_name: x_trans})[0]
    infer_end = time.time()
    
    labels.append(str(label[pred_onx[0]]))
    predicted.append(pred_onx)
    trans_durations.append(trans_end-trans_start)
    infer_durations.append(infer_end - trans_end)

In [141]:
total_time = sum(trans_durations) + sum(infer_durations)
total_transform_time = sum(trans_durations)
total_infer_time = sum(infer_durations)
total_tested = len(predicted)
average_duration_second = total_time / total_tested
average_transform_second = total_transform_time / total_tested
average_infer_second = total_infer_time / total_tested

(530.3888504505157, 36608, 0.014488331797708582)

In [ ]:
batch_size = 32

batch_labels = []
batch_predicted = []
batch_transform_durations = []
batch_infer_durations = []

batch_step = 0
for i in range(0, X_test.shape[0], batch_size):
    x = X_test[i:i+batch_size]
    
    batch_transform_start = time.time()
    x_trans = pipe.transform(x).to_numpy().astype(np.float32)
    batch_transform_end = time.time()
    pred_onx = sess.run([label_name], {input_name: x_trans})[0]
    batch_infer_end = time.time()
    
    batch_labels.extend([str(label[p]) for p in pred_onx.tolist()])
    batch_predicted.extend([p for p in pred_onx.tolist()])
    batch_transform_durations.append(batch_transform_end-batch_transform_start)
    batch_infer_durations.append(batch_infer_end - batch_transform_end)
    batch_step += 1    

In [159]:
batch_total_time = sum(batch_transform_durations) + sum(batch_infer_durations)
batch_transform_total_time = sum(batch_transform_durations)
batch_infer_total_time = sum(batch_infer_durations)
average_batch_duration_second = batch_total_time / batch_step
average_batch_transform_second = batch_transform_total_time / batch_step
average_batch_infer_second = batch_infer_total_time / batch_step
total_time, batch_step, average_batch_duration_second, average_batch_transform_second, average_batch_infer_second
# (16.847609758377075, 1144, 0.014726931606972967) only model

(0.26291346549987793, 1144, 0.00022981946284954363)

In [168]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_absolute_error,
    r2_score,
    mean_squared_error,
)
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


model_type = "classification"
if model_type == "classification":
    metrics = [accuracy_score, precision_score, recall_score, f1_score]
if model_type == "regression":
    metrics = [
        root_mean_squared_error,
        mean_squared_error,
        mean_absolute_error,
        r2_score,
    ]

In [169]:
results = {}
if model_type == "classification":
    results[sess.__class__.__name__] = {
        metric.__name__: (
            metric(y_test, batch_predicted)
            if metric.__name__ == "accuracy_score"
            else metric(y_test, batch_predicted, average='macro')
        )
        for metric in metrics
    }

In [170]:
results

{'InferenceSession': {'accuracy_score': 0.990302666083916,
  'precision_score': 0.9933166673846827,
  'recall_score': 0.718587056899602,
  'f1_score': 0.8014127283691965}}

In [ ]:
def evaluate(test_data_directory: str, preprocess_pipeline: Pipeline) -> Dict:

    classifier = Classifier()

    directory_list = os.listdir(test_data_directory)

    predictions = {}
    predicted = []
    labels = []
    durations = []

    for c in directory_list:
        c_path = os.path.join(c_path, f)
        start = time.time()
        x = classifier.predict_label(data)
        end = time.time()
        duration = end - start
